In [55]:
import pandas as pd
import numpy as np
import z3
from tqdm import tqdm
z3.set_option("parallel.enable", True)

In [56]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 10
DATA_NAME = "harmonic12"
TOPO_NAME = "1"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = int(np.lcm.reduce(task['period']))
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [57]:
# '''
# Fake a dataset that only 2 disjoint traffic with 100ms and 50us

# '''

# # task['period'] = task['period'].apply(lambda x: 10_000_000 if x >= 5_000_000 else 5_000_000)

# task.loc[0] = [8, 15, 100, 100_000_000, (800 + 1000) * 3]
# task.loc[1] = [11, 12, 100, 50_000, (800 + 1000) * 3]
# task = task[:2]

## 1. Model

In [58]:
NUM_WINDOW = 5

In [59]:
s = z3.Solver()
s.set("timeout", time_out*1000)
# s.set("timeout", 4 * 1000) # 4s

Network model

In [60]:
net_var = {}

In [61]:
for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['W'] = NUM_WINDOW
    net_var[row['link']]['N'] = row['q_num']
    net_var[row['link']]['s'] = row['rate']
    net_var[row['link']]['tproc'] = row['t_proc']
    net_var[row['link']]['phi'] = z3.Array(str(row['link']) + '_' + 'phi', z3.IntSort(),z3.IntSort())
    net_var[row['link']]['tau'] = z3.Array(str(row['link']) + '_' + 'tau', z3.IntSort(),z3.IntSort())
    net_var[row['link']]['k'] = z3.Array(str(row['link']) + '_' + 'k', z3.IntSort(),z3.IntSort())
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

Task model

In [62]:
task_var = {}
task_attr = {}

In [63]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [64]:
for i, row in task.iterrows():
    task.loc[i,'route'] = str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0]))))
    task_var.setdefault(i, {})
    task_attr.setdefault(i, {})
    route = eval(task.loc[i, 'route'])
    
    task_attr[i]['C'] = int(row['size'])
    task_attr[i]['T'] = int(row['period'])
    task_attr[i]['L'] = int(row['deadline'])
#     task_attr[i]['J'] = 1000000000 
    
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})
        task_var[i][link] = []
        for j in range(int(LCM / task_attr[i]['T'])):
            task_var[i][link].append(z3.Int('w_' +  str(i) + '_' + str(link) + '_' + str(j)))
        
        task_attr[i][link] = {}
        task_attr[i][link]['l'] = int(task_attr[i]['C']) * 8
        
        
#         task_var[i][link]['phi'] = []
#         task_var[i][link]['p'] =  z3.Int('p_' +  str(i) + '_' + str(link))
#         task_var[i][link]['T'] = np.ceil(row['period'] / net_var[str(link)]['mt'])
#         task_var[i][link]['L'] = np.ceil((row['size'] * 8 / net_var[str(link)]['s']) / net_var[str(link)]['mt'])
#         for j in range(0, int(LCM / row['period'])):
#              task_var[i][link]['phi'].append(z3.Int('phi_' +  str(i) + '_' + str(link) + '_' + str(j)))

## 2. Constraints

In [65]:
for link in net_var:
    for k in range(net_var[link]['W']):
         net_var[link]['tau'] = z3.Store(net_var[link]['tau'], k, net_var[link]['phi'][k])
for i in task_var:
    for link in task_var[i]:
        for j in task_var[i][link]:
            net_var[link]['tau'] = z3.Store(net_var[link]['tau'], j, net_var[link]['tau'][j] + task_attr[i][link]['l'])
# for link in net_var:
#     for k in range(net_var[link]['W']):
#          net_var[link]['tau'] = z3.Store(net_var[link]['tau'], k, net_var[link]['phi'][k])
# for i in task_var:
#     for link in task_var[i]:
#         for j in task_var[i][link]:
#             net_var[link]['tau'] = z3.Store(net_var[link]['tau'], j, net_var[link]['tau'][j] + task_attr[i][link]['l'])

### 2.1 General constraints

Well-deﬁned Windows Constraints [Not used!]

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \forall k \in\left\{1, \ldots, \mathcal{W}^{(a, b)}\right\}: \\
&\left(\phi^{(a, b)}[k] \geq 0\right) \wedge\left(\tau^{(a, b)}[k]<T_{s}\right)
\end{aligned}

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \\
&\left(\phi^{(a, b)}[1] \geq 0\right) \wedge\left(\tau^{(a, b)}\left[\mathcal{W}^{(a, b)}\right]<T_{s}\right)
\end{aligned}

In [66]:
s.add(net_var[link]['phi'][0] >= 0,
        net_var[link]['tau'][-1] < LCM)

In [67]:
# for link in net_var:
#     for k in range(net_var[link]['W']):
#         s.add(net_var[link]['phi'][k] >= 0)
#         s.add(net_var[link]['tau'][k] < LCM)

for link in net_var:
    s.add(net_var[link]['phi'][0] >= 0,
        net_var[link]['tau'][-1] < LCM)

$$\forall(a, b) \in \mathcal{E}: \forall k \in\left\{1, \ldots, \mathcal{W}^{(a, b)}\right\}: 0 \leq \kappa^{(a, b)}[k]<\aleph_{t t}$$

In [68]:
for link in net_var:
    for k in range(net_var[link]['W']):
        s.add(net_var[link]['k'][k] >= 0,
        net_var[link]['k'][k] < net_var[link]['N'])
#s.check()

Stream Instance Constraints

\begin{aligned}
&\forall s_{i} \in \mathcal{S}: \forall(a, b) \in \mathcal{E}: \forall j \in\left[0, \frac{T_{s}}{T_{i}}-1\right]: \\
&\left(\phi^{(a, b)}\left[\omega_{i, j}^{(a, b)}\right] \geq j \times T_{i}\right) \wedge \\
&\left(\tau^{(a, b)}\left[\omega_{i, j}^{(a, b)}\right]<(j+1) \times T_{i}\right)
\end{aligned}

In [69]:
for i in task_var:
    for link in task_var[i]:
        for j in range(int(LCM / task_attr[i]['T'])):
            s.add(net_var[link]['phi'][task_var[i][link][j]] >= j * task_attr[i]['T'],
            net_var[link]['tau'][task_var[i][link][j]] < (j+1) * task_attr[i]['T'])
#s.check()

Zero jitter

\begin{aligned}
&\forall s_{i} \in \mathcal{S}: \forall(a, b) \in \mathcal{E}: \forall j \in\left[0, \frac{T_{s}}{T_{i}}-2\right]: \\
&\left(\phi^{(a, b)}\left[\omega_{i, j+1}^{(a, b)}\right]-\phi^{(a, b)}\left[\omega_{i, j}^{(a, b)}\right]=T_{i}\right) \\
&\forall s_{i} \in \mathcal{S}: \forall(a, b) \in \mathcal{E}: \forall j \in\left[0, \frac{T_{s}}{T_{i}}-1\right]: \\
&\left(\tau^{(a, b)}\left[\omega_{i, j}^{(a, b)}\right]-\phi^{(a, b)}\left[\omega_{i, j}^{(a, b)}\right]=l_{i}^{(a, b)}\right)
\end{aligned}

In [70]:
# for i in task_var:
#     for link in task_var[i]:
#         for j in range(int(LCM / task_attr[i]['T']) - 1):
#             s.add(
#                 net_var[link]['phi'][task_var[i][link][j + 1]] -  
#                 net_var[link]['phi'][task_var[i][link][j]]
#                 == task_attr[i]['T']
#             )
#         for j in range(int(LCM / task_attr[i]['T'])):
#             s.add(
#                 net_var[link]['tau'][task_var[i][link][j]] -  
#                 net_var[link]['phi'][task_var[i][link][j]]
#                 == task_attr[i][link]['l']
#             )
            

Ordered Windows Constraint [Not used!]

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \forall i, j \in\left\{1, \ldots, \mathcal{W}^{(a, b)}\right\}, i \neq j: \\
&\left(\tau^{(a, b)}[i] \leq \phi^{(a, b)}[j]\right) \vee\left(\tau^{(a, b)}[j] \leq \phi^{(a, b)}[i]\right)
\end{aligned}

Less computional exausted one:

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \forall i \in\left\{1, \ldots, \mathcal{W}^{(a, b)}-1\right\}: \\
&\tau^{(a, b)}[i] \leq \phi^{(a, b)}[i+1]
\end{aligned}

In [71]:
for link in net_var:
    for i in range(net_var[link]['W'] - 1):
        s.add(net_var[link]['tau'][i] < net_var[link]['phi'][i + 1])
#s.check()

Frame-to-Window Assignment Constraint

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \forall f_{i, j}^{(a, b)} \in \mathcal{F}^{(a, b)}: \\
&\left(\omega_{i, j}^{(a, b)} \geq 1\right) \wedge\left(\omega_{i, j}^{(a, b)} \leq \mathcal{W}^{(a, b)}\right)
\end{aligned}

In [72]:
for i in task_var:
    for link in task_var[i]:
        for j in task_var[i][link]:
            s.add(0 <= j, j < net_var[link]['W'])

Window Size Constraints

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \forall k \in\left\{1, \ldots, \mathcal{W}^{(a, b)}\right\}: \\
&\tau^{(a, b)}\left\langle k \leftarrow \phi^{(a, b)}[k]\right\rangle
\end{aligned}

In [73]:
# for link in net_var:
#     for k in range(net_var[link]['W']):
#          s.add(net_var[link]['tau'] == z3.Store(net_var[link]['tau'], k, net_var[link]['phi'][k]))

# for link in net_var:
#     for k in range(net_var[link]['W']):
#         s.add(net_var[link]['phi'][k] == z3.Store(net_var[link]['tau'], k, net_var[link]['phi'][k])[k])
# s.check()

\begin{aligned}
&\forall(a, b) \in \mathcal{E}: \forall f_{i, j}^{(a, b)} \in \mathcal{F}^{(a, b)}: \\
&\tau^{(a, b)}\left\langle\omega_{i, j}^{(a, b)} \leftarrow \tau^{(a, b)}\left[\omega_{i, j}^{(a, b)}\right]+l_{i}^{(a, b)}\right\rangle
\end{aligned}

In [74]:
# for i in task_var:
#     for link in task_var[i]:
#         for j in task_var[i][link]:
#             s.add(net_var[link]['tau'] == z3.Store(net_var[link]['tau'], j, net_var[link]['tau'][j] + task_attr[i][link]['l']))
# s.check()

# for i in task_var:
#     for link in task_var[i]:
#         for j in task_var[i][link]:
#             s.add(net_var[link]['tau'][j] + task_attr[i][link]['l'] == z3.Store(net_var[link]['tau'], j, net_var[link]['tau'][j] + task_attr[i][link]['l'])[j])
# # s.check()

Stream Constraint:

\begin{aligned}
&\forall s_{i} \in \mathcal{S}: \forall\left(v_{k}, v_{k+1}\right) \in \mathcal{R}_{i}, k \in\{1, \ldots, n-2\}: \\
&\forall f_{i, j}^{\left(v_{k}, v_{k+1}\right)} \in \mathcal{F}_{i}^{\left(v_{k}, v_{k+1}\right)}: \forall f_{i, j}^{\left(v_{k+1}, v_{k+2}\right)} \in \mathcal{F}_{i}^{\left(v_{k+1}, v_{k+2}\right)}: \\
&\tau^{\left(v_{k}, v_{k+1}\right)}\left[\omega_{i, j}^{\left(v_{k}, v_{k+1}\right)}\right]+\delta \leq \phi^{\left(v_{k+1}, v_{k+2}\right)}\left[\omega_{i, j}^{\left(v_{k+1}, v_{k+2}\right)}\right]
\end{aligned}

In [75]:
for i in task_var:
    hops = list(task_var[i].keys())
    for k, link in enumerate(hops[:-1]):
        for j in range(int(LCM / task_attr[i]['T'])):
            s.add(net_var[link]['tau'][task_var[i][link][j]] + net_var[link]['tproc'] + sync_error <=
                  net_var[hops[k + 1]]['phi'][task_var[i][hops[k + 1]][j]])

### 2.2 IEEE 802.1 Qbv constraints

Stream Isolation Constraint

\begin{aligned}
&\forall k \in\left[0, \frac{T_{s}}{T_{i}}-1\right]: \forall l \in\left[0, \frac{T_{s}}{T_{j}}-1\right]: \\
&\left(\left(\tau^{(a, b)}\left[\omega_{i, k}^{(a, b)}\right]+\delta \leq \phi^{(y, a)}\left[\omega_{j, l}^{(y, a)}\right]\right) \vee\right. \\
&\left.\left(\tau^{(a, b)}\left[\omega_{j, l}^{(a, b)}\right]+\delta \leq \phi^{(x, a)}\left[\omega_{i, k}^{(x, a)}\right]\right)\right) \vee \\
&\left(\kappa^{(a, b)}\left[\omega_{i, k}^{(a, b)}\right] \neq \kappa^{(a, b)}\left[\omega_{j, l}^{(a, b)}\right]\right) \vee\left(\omega_{i, k}^{(a, b)}=\omega_{j, l}^{(a, b)}\right)
\end{aligned}

In [76]:
for i, j in tqdm([(i,j) for i in task_var for j in task_var if i < j]):
    path_i = list(task_var[i].keys())
    path_j = list(task_var[j].keys())
    for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b) 
                        for _x, i_a_b in enumerate(path_i) 
                        for _y, j_a_b in enumerate(path_j) 
                        if i_a_b == j_a_b]:
        for k, l in [(k,l) for k in range(int(LCM / task_attr[i]['T'])) for l in range(int(LCM / task_attr[j]['T']))]:
            s.add(
                z3.Or(
                net_var[a_b]['tau'][task_var[i][a_b][k]] + sync_error <= net_var[y_a]['phi'][task_var[j][y_a][l]],
                net_var[a_b]['tau'][task_var[j][a_b][l]] + sync_error <= net_var[x_a]['phi'][task_var[i][x_a][k]],
                net_var[a_b]['k'][task_var[i][a_b][k]] != net_var[a_b]['k'][task_var[j][a_b][l]],
                task_var[i][a_b][k] == task_var[j][a_b][l]
                )
            )
#s.check()

100%|██████████| 45/45 [00:00<00:00, 1012.27it/s]


Stream End-to-End Latency Constraint

\begin{aligned}
&\forall j \in\left\{0, \cdots, \frac{T_{s}}{T_{i}}-1\right\}: \\
&\forall f_{i, j}^{\left(v_{1}, v_{2}\right)} \in \mathcal{F}_{i}^{\left(v_{1}, v_{2}\right)}, f_{i, j}^{\left(v_{n-1}, v_{n}\right)} \in \mathcal{F}_{i}^{\left(v_{n-1}, v_{n}\right)}: \\
&\tau^{\left(v_{n-1}, v_{n}\right)}\left[\omega_{i, j}^{\left(v_{n-1}, v_{n}\right)}\right]-\phi^{\left(v_{1}, v_{2}\right)}\left[\omega_{i, j}^{\left(v_{1}, v_{2}\right)}\right] \leq L_{i}-\delta
\end{aligned}

In [77]:
for i in task_var:
    _hop_s = list(task_var[i].keys())[0]
    _hop_e = list(task_var[i].keys())[-1]
    for j in range(int(LCM / task_attr[i]['T'])):
        s.add(
            net_var[_hop_e]['tau'][task_var[i][_hop_e][j]] - net_var[_hop_s]['phi'][task_var[i][_hop_s][j]] 
            <= task_attr[i]['L'] - (net_var[link]['tproc'] + sync_error)
        )
#s.check()

Stream jitter constraint


Input jitter 

\begin{aligned}
&\forall j, k \in\left\{0, \cdots, \frac{T_{s}}{T_{i}}-1\right\}: \\
&\forall f_{i, j}^{\left(v_{1}, v_{2}\right)}, f_{i, k}^{\left(v_{1}, v_{2}\right)} \in \mathcal{F}_{i}^{\left(v_{1}, v_{2}\right)}: \\
&\left(\tau^{\left(v_{1}, v_{2}\right)}\left[\omega_{i, j}^{\left(v_{1}, v_{2}\right)}\right]-j \times T_{i}\right)- \\
&\left(\phi^{\left(v_{1}, v_{2}\right)}\left[\omega_{i, k}^{\left(v_{1}, v_{2}\right)}\right]-k \times T_{i}\right)-l_{i}^{\left(v_{1}, v_{2}\right)} \leq J_{i}
\end{aligned}

In [78]:
# for i in tqdm(task_var):
#     _hop_s = list(task_var[i].keys())[0]
#     _hop_e = list(task_var[i].keys())[-1]
#     for j, k in [(j,k) for j in range(int(LCM / task_attr[i]['T'])) for k in range(int(LCM / task_attr[i]['T'])) if j > i]:
#         s.add(
#         (net_var[_hop_s]['tau'][task_var[i][_hop_s][j]] - j * task_attr[i]['T']) - 
#             (net_var[_hop_s]['phi'][task_var[i][_hop_s][k]] - k * task_attr[i]['T']) - 
#             task_attr[i][_hop_s]['l'] <= task_attr[i]['J']
#         )
# # s.check()

Output jitter

\begin{aligned}
&\forall j, k \in\left\{0, \cdots, \frac{T_{s}}{T_{i}}-1\right\}: \\
&\forall f_{i, j}^{\left(v_{n-1}, v_{n}\right)}, f_{i, k}^{\left(v_{n-1}, v_{n}\right)} \in \mathcal{F}_{i}^{\left(v_{n-1}, v_{n}\right)}: \\
&\left(\tau^{\left(v_{n-1}, v_{n}\right)}\left[\omega_{i, j}^{\left(v_{n-1}, v_{n}\right)}\right]-j \times T_{i}\right)- \\
&\left(\phi^{\left(v_{n-1}, v_{n}\right)}\left[\omega_{i, k}^{\left(v_{n-1}, v_{n}\right)}\right]-k \times T_{i}\right)-l_{i}^{\left(v_{n-1}, v_{n}\right)} \leq J_{i}
\end{aligned}

In [79]:
# for i in tqdm(task_var):
#     _hop_s = list(task_var[i].keys())[0]
#     _hop_e = list(task_var[i].keys())[-1]
#     for j, k in [(j,k) for j in range(int(LCM / task_attr[i]['T'])) for k in range(int(LCM / task_attr[i]['T'])) if j > i]:
#         s.add(
#         (net_var[_hop_e]['tau'][task_var[i][_hop_e][j]] - j * task_attr[i]['T']) - 
#             (net_var[_hop_e]['phi'][task_var[i][_hop_e][k]] - k * task_attr[i]['T']) - 
#             task_attr[i][_hop_e]['l'] <= task_attr[i]['J']
#         )
# # s.check()

In [80]:
s.check()

sat

In [81]:
result = s.model()

## Output schedule

In [82]:
## GCL
GCL = []
for link in net_var:
    for i in range(net_var[link]['W']):
        start = result.eval(net_var[link]['phi'][i]).as_long()
        end = result.eval(net_var[link]['tau'][i]).as_long()
        queue = result.eval(net_var[link]['k'][i]).as_long()
        if end > start:
            GCL.append([eval(link), queue, start * macrotick, end * macrotick, LCM * macrotick])
            
## Offset
OFFSET = []
for i in task_var:
    link = list(task_var[i].keys())[0]
    for ins_id, ins_window in enumerate(list(task_var[i].values())[0]):
        offset = result.eval(net_var[link]['phi'][ins_window]).as_long()
        OFFSET.append([i, ins_id, (task.loc[i,'period'] - offset) * macrotick])

ROUTE = []
for i, row in task.iterrows():
    route = eval(str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0])))))
    for h, v in enumerate(route[:-1]):
        ROUTE.append(
            [i, (v, route[h + 1])]
        )

QUEUE = []
for i in task_var:
    for link in task_var[i]:
        for ins_id, ins_window in enumerate(task_var[i][link]):
            QUEUE.append(
                [i, ins_id, link, result.eval(net_var[link]['k'][ins_window]).as_long()]
            )

In [83]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTAS2018-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTAS2018-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTAS2018-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTAS2018-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

In [84]:
info = s.statistics()
info.time

8.115

In [85]:
# Z3 megabytes
info.max_memory

116.13